In [1]:
import os

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader

from swin import SwinTransformer, _patch_expanding_pad


In [2]:
*A, B= 1, 2
A


[1]

In [3]:
x = torch.tensor([
    [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
    [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
    [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
    [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
])

print(torch.cat((x, x), dim=-1))

x = torch.stack((x, x, x, x))

# x = x.reshape(2, 2, -1)
x = x.unsqueeze(0)

x = _patch_expanding_pad(x)

print(x.shape)

print(x[0][1][3])


tensor([[1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 1, 1, 1, 1, 2, 2, 2, 2,
         3, 3, 3, 3, 4, 4, 4, 4],
        [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 1, 1, 1, 1, 2, 2, 2, 2,
         3, 3, 3, 3, 4, 4, 4, 4],
        [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 1, 1, 1, 1, 2, 2, 2, 2,
         3, 3, 3, 3, 4, 4, 4, 4],
        [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 1, 1, 1, 1, 2, 2, 2, 2,
         3, 3, 3, 3, 4, 4, 4, 4]])
torch.Size([1, 8, 8, 4])
tensor([4, 4, 4, 4])


In [4]:
patch_size = [4, 4]
embed_dim = 64
depths = [1, 1, 1]
num_heads = [4, 4, 4]
window_size = [4, 4]
num_classes = 1


swin = SwinTransformer(patch_size, embed_dim, depths, num_heads, window_size, num_classes=num_classes)

IMG = 64
x = torch.randn((1, 3, IMG, IMG))
print(x.shape)

y = swin(x)
print(y.shape)

print(swin)


torch.Size([1, 3, 64, 64])
torch.Size([1, 10, 64, 64])
SwinTransformer(
  (patching): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(4, 4))
    (1): Permute()
    (2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  )
  (encoder): ModuleList(
    (0): Sequential(
      (0): SwinTransformerBlockV2(
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (attn): ShiftedWindowAttentionV2(
          (qkv): Linear(in_features=64, out_features=192, bias=True)
          (proj): Linear(in_features=64, out_features=64, bias=True)
          (cpb_mlp): Sequential(
            (0): Linear(in_features=2, out_features=512, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=512, out_features=4, bias=False)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (0): Linear(in_features=64, out_feature